In [1]:
import pandas as pd 
import configparser
import os
import matplotlib.pyplot as plt
import numpy as np

# Configuration
config_file = "c:\\Users\\shaik.ms.1\\OneDrive - Procter and Gamble\\Documents\\Projects\\Personal\\nocturnal_flight_calling\\config\\config.ini"
cp = configparser.ConfigParser()
cp.read(config_file)

# Environment 
env = cp["Env"]["env"]
input_path ="c:\\Users\\shaik.ms.1\\OneDrive - Procter and Gamble\\Documents\\Projects\\Personal\\nocturnal_flight_calling\\" +cp["Input"]["path"]
chicago_collision = input_path + cp["Input"]["chicago_collision"]
flight_call = input_path + cp["Input"]["flight_call"]
light_levels = input_path + cp["Input"]["light_levels"]

# Read data as JSON file 
chicago_collision_df = pd.read_json(chicago_collision)
flight_call_df = pd.read_json(flight_call)
light_levels_df = pd.read_json(light_levels)

rename_cols ={"Species":"Genus","Family":"Species","Collisions":"Family", "Call":"Flight Call","Flight":"Collisions"}
flight_call_df.rename(columns = rename_cols,inplace=True)
flight_call_df["Flight Call"]= flight_call_df["Flight Call"].str.lower()
#flight_call_df = flight_call_df[flight_call_df["Flight Call"]!="rare"]

print(chicago_collision_df.head())
print(flight_call_df.head())
print(light_levels_df.head())


print(flight_call_df.describe())
print(light_levels_df.describe())


# Clean Data 

print(chicago_collision_df.sort_values(by=['Date']))
print(pd.unique(flight_call_df["Flight Call"]))


        Genus  Species       Date Locality
0  Ammodramus  nelsoni 1982-10-03       MP
1  Ammodramus  nelsoni 1984-05-21      CHI
2  Ammodramus  nelsoni 1984-05-25       MP
3  Ammodramus  nelsoni 1985-10-08       MP
4  Ammodramus  nelsoni 1986-09-10       MP
         Genus      Species         Family  Collisions Flight Call Habitat  \
0  Zonotrichia   albicollis  Passerellidae       10133         yes  Forest   
1        Junco     hyemalis  Passerellidae        6303         yes    Edge   
2    Melospiza      melodia  Passerellidae        5124         yes    Edge   
3    Melospiza    georgiana  Passerellidae        4910         yes    Open   
4      Seiurus  aurocapilla      Parulidae        4580         yes  Forest   

  Stratum  
0   Lower  
1   Lower  
2   Lower  
3   Lower  
4   Lower  
        Date  Light Score 
0 2010-09-26           4.0
1 2015-05-07           6.0
2 2012-03-07           NaN
3 2000-09-18          15.0
4 2000-11-10          15.0
         Collisions
count     96.000000

In [2]:
print(chicago_collision_df.count())
print(chicago_collision_df.info())
print(chicago_collision_df.describe())
chicago_collision_df.sort_values(by = ["Date"],ascending=False)

chicago_collision_df['Month'] = pd.DatetimeIndex(chicago_collision_df['Date']).month
chicago_collision_df['Season'] = chicago_collision_df['Month'].map({3:"spring",4:"spring",5:"spring",8:"autumn",9:"autumn",10:"autumn",11:"autumn"})
chicago_collision_df.drop(labels=["Month"], axis=1, inplace= True)
# Standardize it to lower for joining with flight call data 
chicago_collision_df["Species"]= chicago_collision_df["Species"].str.lower()
chicago_collision_df["Genus"]= chicago_collision_df["Genus"].str.lower()

def corrected_genus(genus,species):
    if(genus == "ammodramus" and species == "henslowii"):
        return "centronyx"
    elif(genus == "ammodramus" and species == "nelsoni" ):
        return "ammospiza"
    elif(genus == "ammodramus" and species == "leconteii" ):
        return "ammospiza"
    else:
        return genus

chicago_collision_df["Genus"] = chicago_collision_df.apply(lambda x: corrected_genus(x["Genus"],x["Species"]), axis= 1)


Genus       69784
Species     69784
Date        69784
Locality    69784
dtype: int64
<class 'pandas.core.frame.DataFrame'>
Int64Index: 69784 entries, 0 to 69783
Data columns (total 4 columns):
Genus       69784 non-null object
Species     69784 non-null object
Date        69784 non-null datetime64[ns]
Locality    69784 non-null object
dtypes: datetime64[ns](1), object(3)
memory usage: 2.7+ MB
None
            Genus     Species                 Date Locality
count       69784       69784                69784    69784
unique         43          84                 5318        2
top     Melospiza  albicollis  2013-10-08 00:00:00       MP
freq        12063       10133                  364    36315
first         NaN         NaN  1978-09-15 00:00:00      NaN
last          NaN         NaN  2016-11-30 00:00:00      NaN


In [3]:
print(flight_call_df.count())
print(flight_call_df.info())
print(flight_call_df.describe())

flight_call_df.sort_values(by = ["Collisions"],ascending=False)

flight_call_df["Flight Call"] = flight_call_df["Flight Call"].map({"yes":"yes","no":"no", "rare":"no"})
flight_call_df["Stratum"] = flight_call_df["Stratum"].str.lower().replace("\t","")
flight_call_df["Stratum"] = flight_call_df["Stratum"].str.replace("\t","")
flight_call_df["Habitat"] = flight_call_df["Habitat"].str.lower()
flight_call_df["Species"]= flight_call_df["Species"].str.lower()
flight_call_df["Genus"]= flight_call_df["Genus"].str.lower()
flight_call_df.drop_duplicates(inplace=True)
print("Habitat", pd.unique(flight_call_df["Habitat"]))
print("Stratum", pd.unique(flight_call_df["Stratum"]))
print("Flight Call", pd.unique(flight_call_df["Flight Call"]))

Genus          96
Species        96
Family         96
Collisions     96
Flight Call    96
Habitat        96
Stratum        96
dtype: int64
<class 'pandas.core.frame.DataFrame'>
Int64Index: 96 entries, 0 to 95
Data columns (total 7 columns):
Genus          96 non-null object
Species        96 non-null object
Family         96 non-null object
Collisions     96 non-null int64
Flight Call    96 non-null object
Habitat        96 non-null object
Stratum        96 non-null object
dtypes: int64(1), object(6)
memory usage: 6.0+ KB
None
         Collisions
count     96.000000
mean     807.927083
std     1602.275587
min        1.000000
25%       17.500000
50%      177.000000
75%      742.000000
max    10133.000000
Habitat ['forest' 'edge' 'open']
Stratum ['lower' 'upper']
Flight Call ['yes' 'no']


,Genus,Species,Family,Collisions,Flight Call,Habitat,Stratum
0,zonotrichia,albicollis,Passerellidae,10133,yes,forest,lower
1,junco,hyemalis,Passerellidae,6303,yes,edge,lower
2,melospiza,melodia,Passerellidae,5124,yes,edge,lower
3,melospiza,georgiana,Passerellidae,4910,yes,open,lower
4,seiurus,aurocapilla,Parulidae,4580,yes,forest,lower
...,...,...,...,...,...,...,...
86,setophaga,cerulea,Parulidae,3,yes,forest,upper
87,vireo,gilvus,Vireonidae,2,no,forest,upper
88,zonotrichia,querula,Passerellidae,2,yes,edge,lower
89,lanius,excubitor,Laniidae,1,no,open,upper


In [4]:
print(light_levels_df.count())
print(light_levels_df.info())
print(light_levels_df.describe())

Date            3236
Light Score     3236
dtype: int64
<class 'pandas.core.frame.DataFrame'>
Int64Index: 3251 entries, 0 to 3250
Data columns (total 2 columns):
Date            3236 non-null datetime64[ns]
Light Score     3236 non-null float64
dtypes: datetime64[ns](1), float64(1)
memory usage: 76.2 KB
None
       Light Score 
count   3236.000000
mean      10.338690
std        5.279244
min        3.000000
25%        4.000000
50%       11.000000
75%       15.000000
max       17.000000


In [5]:

c = pd.merge(chicago_collision_df, flight_call_df,  how='left', on=['Genus','Species'])
c

,Genus,Species,Date,Locality,Season,Family,Collisions,Flight Call,Habitat,Stratum
0,ammospiza,nelsoni,1982-10-03,MP,autumn,Passerellidae,55,yes,open,lower
1,ammospiza,nelsoni,1984-05-21,CHI,spring,Passerellidae,55,yes,open,lower
2,ammospiza,nelsoni,1984-05-25,MP,spring,Passerellidae,55,yes,open,lower
3,ammospiza,nelsoni,1985-10-08,MP,autumn,Passerellidae,55,yes,open,lower
4,ammospiza,nelsoni,1986-09-10,MP,autumn,Passerellidae,55,yes,open,lower
...,...,...,...,...,...,...,...,...,...,...
77556,melospiza,melodia,2016-10-19,CHI,autumn,Passerellidae,5124,yes,edge,lower
77557,melospiza,melodia,2016-10-30,CHI,autumn,Passerellidae,5124,yes,edge,lower
77558,zonotrichia,querula,2007-10-15,MP,autumn,Passerellidae,2,yes,edge,lower
77559,zonotrichia,querula,2008-10-16,MP,autumn,Passerellidae,2,yes,edge,lower


In [6]:
grou= c[~c["Family"].notnull()][["Genus","Species","Locality"]].groupby(by=["Genus","Species"]).agg("count")

print(grou[['Locality']].sum())
c[~c["Family"].notnull()][["Genus","Species","Family"]].count

c

Locality    0
dtype: int64


,Genus,Species,Date,Locality,Season,Family,Collisions,Flight Call,Habitat,Stratum
0,ammospiza,nelsoni,1982-10-03,MP,autumn,Passerellidae,55,yes,open,lower
1,ammospiza,nelsoni,1984-05-21,CHI,spring,Passerellidae,55,yes,open,lower
2,ammospiza,nelsoni,1984-05-25,MP,spring,Passerellidae,55,yes,open,lower
3,ammospiza,nelsoni,1985-10-08,MP,autumn,Passerellidae,55,yes,open,lower
4,ammospiza,nelsoni,1986-09-10,MP,autumn,Passerellidae,55,yes,open,lower
...,...,...,...,...,...,...,...,...,...,...
77556,melospiza,melodia,2016-10-19,CHI,autumn,Passerellidae,5124,yes,edge,lower
77557,melospiza,melodia,2016-10-30,CHI,autumn,Passerellidae,5124,yes,edge,lower
77558,zonotrichia,querula,2007-10-15,MP,autumn,Passerellidae,2,yes,edge,lower
77559,zonotrichia,querula,2008-10-16,MP,autumn,Passerellidae,2,yes,edge,lower


In [7]:
print(flight_call_df[flight_call_df['Species']=='henslowii'])

chicago_collision_df[chicago_collision_df['Species']=='henslowii'].sort_values(by=["Date"],ascending=False)

        Genus    Species         Family  Collisions Flight Call Habitat  \
77  centronyx  henslowii  Passerellidae           9         yes    open   

   Stratum  
77   lower  


,Genus,Species,Date,Locality,Season
61,centronyx,henslowii,2015-04-29,CHI,spring
60,centronyx,henslowii,2012-05-25,CHI,spring
59,centronyx,henslowii,2010-05-11,CHI,spring
58,centronyx,henslowii,2005-04-19,CHI,spring
57,centronyx,henslowii,1988-05-13,MP,spring
56,centronyx,henslowii,1986-05-06,MP,spring
55,centronyx,henslowii,1984-04-28,MP,spring
54,centronyx,henslowii,1984-04-27,MP,spring
53,centronyx,henslowii,1983-10-06,MP,autumn


In [18]:
d = c.fillna("missing").groupby(by=["Family","Genus","Species","Flight Call"]).agg(collisions=('Species', 'count'), collisions_days =('Date','nunique') ).sort_values(by= "collisions", ascending = False)
d.reset_index(inplace = True)
print(d)
c[~c["Family"].notnull()].fillna("missing").groupby(by=["Family","Genus","Species","Flight Call"]).agg(collisions=('Species', 'count'), collisions_days =('Date','nunique') ).sort_values(by= "collisions", ascending = False)

           Family        Genus     Species Flight Call  collisions  \
0   Passerellidae  zonotrichia  albicollis         yes       10133   
1   Passerellidae    melospiza   georgiana         yes        9820   
2   Passerellidae        junco    hyemalis         yes        6303   
3      Certhiidae      certhia   americana         yes        5352   
4   Passerellidae    melospiza     melodia         yes        5124   
..            ...          ...         ...         ...         ...   
86     Vireonidae        vireo      gilvus          no           4   
87      Parulidae    setophaga     cerulea         yes           3   
88  Passerellidae  zonotrichia     querula         yes           2   
89       Laniidae       lanius   excubitor          no           1   
90   Cardinalidae    passerina    caerulea         yes           1   

    collisions_days  
0              1852  
1              1550  
2              1686  
3               815  
4              1156  
..              ...  
86   

,,,,collisions,collisions_days
Family,Genus,Species,Flight Call,,


In [17]:
c["Family"].unique()

array(['Passerellidae', 'Turdidae', 'Certhiidae', 'Troglodytidae',
       'Tyrannidae', 'Parulidae', 'Mimidae', 'Icteriidae', 'Icteridae',
       'Laniidae', 'Cardinalidae', 'Polioptilidae', 'Regulidae',
       'Sittidae', 'Vireonidae'], dtype=object)

In [9]:
light_levels_df.sort_values(by=["Date"],ascending=False)

,Date,Light Score
2136,2018-05-26,4.0
1950,2018-05-25,11.0
2735,2018-05-24,11.0
3236,2018-05-23,11.0
2668,2018-05-22,14.0
...,...,...
2596,NaT,15.0
2742,NaT,3.0
2984,NaT,6.0
3030,NaT,14.0


In [10]:
x = pd.merge(c, light_levels_df,  how='left', on=['Date'])
print(len(pd.unique(x[x["Light Score "].notnull() & x["Family"].notnull()].sort_values(by=["Date"])["Date"])))

x['Month'] = pd.DatetimeIndex(x['Date']).month
x['Season'] = x['Month'].map({3:"spring",4:"spring",5:"spring",8:"autumn",9:"autumn",10:"autumn",11:"autumn"})


x.drop(labels = ["Month"], axis =1)

2320


,Genus,Species,Date,Locality,Season,Family,Collisions,Flight Call,Habitat,Stratum,Light Score
0,ammospiza,nelsoni,1982-10-03,MP,autumn,Passerellidae,55,yes,open,lower,NaN
1,ammospiza,nelsoni,1984-05-21,CHI,spring,Passerellidae,55,yes,open,lower,NaN
2,ammospiza,nelsoni,1984-05-25,MP,spring,Passerellidae,55,yes,open,lower,NaN
3,ammospiza,nelsoni,1985-10-08,MP,autumn,Passerellidae,55,yes,open,lower,NaN
4,ammospiza,nelsoni,1986-09-10,MP,autumn,Passerellidae,55,yes,open,lower,NaN
...,...,...,...,...,...,...,...,...,...,...,...
80001,melospiza,melodia,2016-10-19,CHI,autumn,Passerellidae,5124,yes,edge,lower,10.0
80002,melospiza,melodia,2016-10-30,CHI,autumn,Passerellidae,5124,yes,edge,lower,3.0
80003,zonotrichia,querula,2007-10-15,MP,autumn,Passerellidae,2,yes,edge,lower,14.0
80004,zonotrichia,querula,2008-10-16,MP,autumn,Passerellidae,2,yes,edge,lower,4.0


In [11]:

df["Habitat"] = flight_call_df["Habitat"].str.lower()x_count.set_index(["Family","Genus","Species"],inplace = True )
x_count
x_count["flag"] = x_count["Flight Call"].apply(lambda x: 1  if x == "yes" else 0)
#x_count.drop(["Collisions"])
x_count[["collision_sqrt","flag"]].sort_values(by =["collision_sqrt"],ascending = False).iloc[0:10].plot.barh(y="collision_sqrt")

SyntaxError: invalid syntax (<ipython-input-11-b5428fc4f9be>, line 1)

Flight Call  Collisions  collision_sqrt
Genus       Species                                               
Ammodramus  savannarum             yes         106       10.295630
Cardellina  canadensis             yes         245       15.652476
            pusilla                yes         190       13.784049
Catharus    fuscescens             yes         743       27.258026
            guttatus               yes        3840       61.967734
...                                ...         ...             ...
Vireo       gilvus                  no           6        2.449490
            olivaceus               no         137       11.704700
            philadelphicus          no          12        3.464102
Zonotrichia albicollis             yes       10623      103.067939
            leucophrys             yes        1131       33.630343

[79 rows x 3 columns]